# Getting the Data
Here we will download and prepare data published by Zhang et al. 2019 (https://github.com/bmmi/denoising-fluorescence).

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pickle

import pn2v.utils
import pn2v.histNoiseModel
from tifffile import imread

### Download the data

In [10]:
import os

# os.system("rm -r data")
path = './FMD'
outname = 'TwoPhoton_MICE'
if not os.path.isdir(path):
    os.mkdir(path)

fileid='1lhsFAlXsXk26yqHzT0_-3R8MUb7G0NVa'
command= f"wget --load-cookies /tmp/cookies.txt \"https://docs.google.com/uc?export=download&confirm=$(wget --quiet " \
         f"--save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate " \
         f"\'https://docs.google.com/uc?export=download&id={fileid}\' -O- | sed -rn \'s/.*confirm=([0-9A-Za-z_]+).*" \
         f"/\\1\\n/p\')&id={fileid}\" -O {path}/{outname}.tar && rm -rf /tmp/cookies.txt"
print (os.system(command) )

--2023-05-08 09:13:45--  https://docs.google.com/uc?export=download&confirm=t&id=1lhsFAlXsXk26yqHzT0_-3R8MUb7G0NVa
Resolving docs.google.com (docs.google.com)... 172.217.168.206, 2a00:1450:400e:80c::200e
Connecting to docs.google.com (docs.google.com)|172.217.168.206|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-04-04-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/t3nhp6jobundcq0e6q5mij17glbpnm0f/1683526200000/17828459380737346642/*/1lhsFAlXsXk26yqHzT0_-3R8MUb7G0NVa?e=download&uuid=8ff28c5b-e5ab-434c-bdb4-a4ea45639530 [following]
--2023-05-08 09:13:46--  https://doc-04-04-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/t3nhp6jobundcq0e6q5mij17glbpnm0f/1683526200000/17828459380737346642/*/1lhsFAlXsXk26yqHzT0_-3R8MUb7G0NVa?e=download&uuid=8ff28c5b-e5ab-434c-bdb4-a4ea45639530
Resolving doc-04-04-docs.googleusercontent.com (doc-04-04-docs.googleusercontent.com)... 142.251.39.97, 2a00:1450:

0


......... .......... .......... .......... .......... 99% 25.0M 0s
844600K .......... .......... .......... .......... .......... 99% 56.6M 0s
844650K .......... .......... .......... .......... .......... 99% 36.8M 0s
844700K .......... .......... .......... .......... .......... 99% 63.1M 0s
844750K .......... .......... .......... .......... .......... 99% 40.3M 0s
844800K .......... .......... .......... .......... .......... 99% 37.3M 0s
844850K .......... .......... .......... .......... .......... 99% 44.5M 0s
844900K .......... .......... .......... .......... .......... 99% 42.2M 0s
844950K .......... .......... .......... .......... .......... 99% 57.0M 0s
845000K .......... .......... .......... .......... .......... 99% 78.8M 0s
845050K .......... .......... .......... .......... .......... 99% 31.7M 0s
845100K .......... .......... .......... .......... .......... 99% 36.2M 0s
845150K .......... .......... .......... .......... .......... 99% 35.8M 0s
845200K .......... ..

In [15]:
import tarfile,sys
outname = 'TwoPhoton_MICE'

tar = tarfile.open(path +f'/{outname}.tar')
tar.extractall(path)
tar.close()

In [16]:
import numpy as np
import glob
from skimage import io
import matplotlib.pyplot as plt

def loadFOV(number, path):
    path=path+str(number)+"/*.png"
    files = glob.glob(path)
    data=[]
    print(path)
    for f in files:
        
        im_b = np.array(io.imread(f))
        data.append(im_b)
    data = np.array(data)
    return data
    
def loadFOVSingle(number, path):
    path=path+str(number)+"/*.png"
    files = glob.glob(path)
    data=[]
    print(path)
    f=files[0]
    im_b = np.array(io.imread(f))
    data.append(im_b)
    data = np.array(data)
    return data
    

In [19]:
dataRaw=None
dataGT=None

dataTestRaw=None
dataTestGT=None

outname = 'TwoPhoton_MICE'


path = f'FMD/{outname}/raw/'
for i in range(1,20+1):
    if i==19:    # FOV 19 is reserved for testing
        dataTestRaw=loadFOV(i,path).astype(np.float32)
    else:
        newa=loadFOV(i,path).astype(np.float32)
        if dataRaw is None:
            dataRaw=newa
        else:
            print(dataRaw.shape, newa.shape)
            dataRaw=np.concatenate((dataRaw,newa),axis=0)
    print(dataRaw.shape)

dataRaw=np.array(dataRaw)    
np.save(path+'/training_raw.npy', dataRaw)
np.save(path+'/test_raw.npy', dataTestRaw)

path = f'FMD/{outname}/gt/'
for i in range(1,20+1):
    if i==19:   # FOV 19 is reserved for testing
        dataTestGT=loadFOV(i,path).astype(np.float32)
    else:
        newa=loadFOV(i,path).astype(np.float32)
        if dataGT is None:
            dataGT=newa
        else:
            print(dataRaw.shape, newa.shape)
            dataGT=np.concatenate((dataGT,newa),axis=0)
        print(dataRaw.shape)
    
dataGT=np.array(dataGT)
dataRaw=np.array(dataRaw)
np.save(path+'/training_gt.npy', dataGT)
np.save(path+'/test_gt.npy', dataTestGT)

FMD/TwoPhoton_MICE/raw/1/*.png
(50, 512, 512)
FMD/TwoPhoton_MICE/raw/2/*.png
(50, 512, 512) (50, 512, 512)
(100, 512, 512)
FMD/TwoPhoton_MICE/raw/3/*.png
(100, 512, 512) (50, 512, 512)
(150, 512, 512)
FMD/TwoPhoton_MICE/raw/4/*.png
(150, 512, 512) (50, 512, 512)
(200, 512, 512)
FMD/TwoPhoton_MICE/raw/5/*.png
(200, 512, 512) (50, 512, 512)
(250, 512, 512)
FMD/TwoPhoton_MICE/raw/6/*.png
(250, 512, 512) (50, 512, 512)
(300, 512, 512)
FMD/TwoPhoton_MICE/raw/7/*.png
(300, 512, 512) (50, 512, 512)
(350, 512, 512)
FMD/TwoPhoton_MICE/raw/8/*.png
(350, 512, 512) (50, 512, 512)
(400, 512, 512)
FMD/TwoPhoton_MICE/raw/9/*.png
(400, 512, 512) (50, 512, 512)
(450, 512, 512)
FMD/TwoPhoton_MICE/raw/10/*.png
(450, 512, 512) (50, 512, 512)
(500, 512, 512)
FMD/TwoPhoton_MICE/raw/11/*.png
(500, 512, 512) (50, 512, 512)
(550, 512, 512)
FMD/TwoPhoton_MICE/raw/12/*.png
(550, 512, 512) (50, 512, 512)
(600, 512, 512)
FMD/TwoPhoton_MICE/raw/13/*.png
(600, 512, 512) (50, 512, 512)
(650, 512, 512)
FMD/TwoPhoton_M